In [0]:
%python
# Define the file path
filePath = "/Volumes/workspace/globalretail/products"

# Read the JSON file into a DataFrame
df = spark.read.json(filePath)

# Display the DataFrame
display(df)

In [0]:
from pyspark.sql.functions import current_timestamp
df_new = df.withColumn("ingestion_timestamp", current_timestamp())
display(df_new)

In [0]:
spark.sql("use globalretail_bronze")
df_new.write.format("delta").mode("append").saveAsTable("bronze_products")

In [0]:
spark.sql("select * from bronze_products limit 100").show()

In [0]:
%python
# Create the volume if it does not exist
spark.sql("""
CREATE VOLUME IF NOT EXISTS workspace.globalretail.archive
COMMENT 'Archive volume for products data'
""")

import datetime

# Define the source and archive paths
source_file = "/Volumes/workspace/globalretail/products/"
archive_folder = "/Volumes/workspace/globalretail/archive"
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
archive_file = f"{archive_folder}/products_{timestamp}.csv"

# Move the file from the original path to the new archive path
dbutils.fs.mv(source_file, archive_file, recurse=True)

# Print the new archive file path
print(f"File archived at: {archive_file}")